In [45]:
%matplotlib inline
%autosave 0
import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
import plotly.express as px

Autosave disabled


In [77]:
eiaLogo = "https://upload.wikimedia.org/wikipedia/commons/1/1f/Energy_Information_Administration_logo.svg"
eiaLg = [dict(source = eiaLogo,
                       x=1.07, y=-0.1,
                       sizex=0.2, sizey=0.2,
                       xanchor="center", yanchor="bottom")]
eiaLg2 = [dict(source = eiaLogo,
                        x=1.03, y=1.1,
                        sizex=0.1, sizey=0.1,
                        xanchor="center", yanchor="bottom")]

In [2]:
rs = [('https://www.eia.gov/opendata/qb.php?sdid=NG.N3020US2.M', 'comDemand')      
     ,('https://www.eia.gov/opendata/qb.php?sdid=NG.N3010US2.M', 'resDemand')      
     ,('https://www.eia.gov/opendata/qb.php?sdid=NG.N3035US2.M', 'indDemand')
     ,('https://www.eia.gov/opendata/qb.php?sdid=NG.N3025US2.M', 'vehDemand')
     ,('https://www.eia.gov/opendata/qb.php?sdid=NG.N9160US2.M', 'l&pDemand')
     ,('https://www.eia.gov/opendata/qb.php?sdid=NG.N9170US2.M', 'pplDemand')
     ,('https://www.eia.gov/opendata/qb.php?sdid=NG.N3045US2.M', 'PowerGen')
     #,('https://www.eia.gov/opendata/qb.php?sdid=NG.N5050US2.M', 'strInjections')       
     ,('https://www.eia.gov/opendata/qb.php?sdid=NG.NGM_EPG0_ENC_NUS-Z00_MMCF.M', 'cgsExports') 
     ,('https://www.eia.gov/opendata/qb.php?sdid=NG.N9132US2.M', 'pplExports')
     ,('https://www.eia.gov/opendata/qb.php?sdid=NG.N9133US2.M', 'lngExports')]

storageSeason = { 1: 'wd', 2: 'wd', 3: 'wd', 11: 'wd', 12: 'wd',
                  4: 'ij', 5: 'ij', 6: 'ij', 7: 'ij', 8: 'ij', 9: 'ij', 10: 'ij'}
sData = pd.DataFrame()

for r, name in rs:
    data = pd.read_html(r)
    sName = data[0]['Series Name'][0]
    
    df = data[0][['Period','Value']][:]
    df['Period']  = pd.to_datetime(df.Period, format='%Y%m', errors='coerce').dropna()
    df.columns = ['Period', name]
    df = df.set_index('Period')
    df.drop(df[df.index < '2001-01-01'].index, inplace = True)
    df['Year'] = pd.DatetimeIndex(df.index).year
    df['Month'] = pd.DatetimeIndex(df.index).month
    df['sYear']  = df['Year'].shift(-3).fillna(0).astype(int).astype(str)
    df['Season'] = df['Month'].apply(lambda x: storageSeason[x]) + df['sYear']
    df.drop(df[df['Season'] == 'wd0'].index, inplace = True)

    sdf = df[[name,'Season']].groupby(['Season'],as_index=False)[name].sum()
    sdf = sdf.set_index('Season')
    print(len(sdf), name,'=', sName)
    if sData.empty:
        sData = sdf
    else:
        sData = sData.join(sdf)
        
sData *= 1000000    

42 comDemand = Natural Gas Deliveries to Commercial Consumers (Including Vehicle Fuel through 1996) in the U.S., Monthly
42 resDemand = U.S. Natural Gas Residential Consumption, Monthly
42 indDemand = U.S. Natural Gas Industrial Consumption, Monthly
42 vehDemand = U.S. Natural Gas Vehicle Fuel Consumption, Monthly
42 l&pDemand = U.S. Natural Gas Lease and Plant Fuel Consumption, Monthly
42 pplDemand = U.S. Natural Gas Pipeline & Distribution Use, Monthly
42 PowerGen = U.S. Natural Gas Deliveries to Electric Power Consumers, Monthly
22 cgsExports = Compressed U.S. Natural Gas Exports, Monthly
42 pplExports = U.S. Natural Gas Pipeline Exports, Monthly
42 lngExports = Liquefied U.S. Natural Gas Exports, Monthly


In [82]:
fig = px.bar(sData,  title="U.S. NG demand by storage seasons")
fig.update_layout(xaxis_title='storage season year', yaxis_title='demand in cubic feet',legend={'traceorder': 'reversed'})
fig.layout.images = eiaLg
fig.show()

In [6]:
ijSeason = sData[sData.index.str.contains('ij')]
wdSeason = sData[sData.index.str.contains('wd')]

In [83]:
fig = px.bar(ijSeason, title="U.S. NG demand during injection seasons")
fig.update_layout(xaxis_title='injection season year', yaxis_title='demand in cubic feet',legend={'traceorder': 'reversed'})
fig.layout.images = eiaLg
fig.show()

In [81]:
fig = px.bar(wdSeason, title="U.S. NG demand during withdrawal seasons")
fig.update_layout(xaxis_title='injection season year', yaxis_title='demand in cubic feet',legend={'traceorder': 'reversed'})
fig.layout.images = eiaLg
fig.show()

In [8]:
ijSeasonAll = ijSeason.sum(axis=1)
wdSeasonAll = wdSeason.sum(axis=1)

In [78]:
fig = px.bar(x = ijSeasonAll.index, y = ijSeasonAll,
             title="U.S. NG demand during injection seasons",
             labels={'y': "demand in cubic feet", 'x': ' injection season year'})
fig.layout.showlegend = True  
fig.layout.images = eiaLg2
fig.show()

In [79]:
fig = px.bar(x = wdSeasonAll.index, y = wdSeasonAll,
             title="U.S. NG demand during withdrawal seasons",
             labels={'y': "demand in cubic feet", 'x': ' withdrawal season year'})
            
fig.layout.showlegend = False
fig.layout.images = eiaLg2
fig.show()

In [80]:
fig = px.bar(x = wdSeasonAll.index, y = wdSeasonAll.diff(), 
             title="Y/Y change in NG demand during withdrawal season", 
             labels={'y': "cubic feet", 'x': ' storage season & year'})
             
fig.layout.showlegend = False
fig.layout.images = eiaLg2
fig.show()